In [1]:
import sys
import logging
import pickle

import numpy as np
import matplotlib.pyplot as plt
from joblib import hash, dump, load
import os

from deer.default_parser import process_args
from deer.agent import NeuralAgent
from deer.learning_algos.CRAR_torch import CRAR
from figure8_env import MyEnv as figure8_env
import deer.experiment.base_controllers as bc

from deer.policies import EpsilonGreedyPolicy, FixedFigure8Policy

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
figure8_give_rewards = True
nn_yaml = 'network.yaml'
high_dim_obs = True #False
internal_dim = 10
fname = '2d_obs'
set_network = None #['expanded_tcm', 15, True]

In [3]:
class Defaults:
    # ----------------------
    # Setup Parameters (copied for convenience)
    # ----------------------
    figure8_give_rewards = figure8_give_rewards
    nn_yaml = nn_yaml
    high_dim_obs = high_dim_obs
    internal_dim = internal_dim
    fname = fname
    
    # ----------------------
    # Experiment Parameters
    # ----------------------
    steps_per_epoch = 500 #5000
    epochs = 50
    steps_per_test = 1000
    period_btw_summary_perfs = 1

    # ----------------------
    # Temporal Processing Parameters
    # ----------------------
    nstep = 2 #15 #20
    nstep_decay = 0.8
    expand_tcm = True
    encoder_type = 'regular'
    
    # ----------------------
    # Environment Parameters
    # ----------------------
    frame_skip = 2
    show_rewards = False

    # ----------------------
    # DQN Agent parameters:
    # ----------------------
    learning_rate = 1*1E-4
    learning_rate_decay = 1.0
    discount = 0.9
    epsilon_start = 1.0
    epsilon_min = 1.0
    epsilon_decay = 1000
    update_frequency = 1
    replay_memory_size = 100000 #50000
    batch_size = 64
    freeze_interval = 1000
    deterministic = False
    
    # ----------------------
    # Learning algo parameters
    # ----------------------
    # T, entropy_neighbor, entropy_random, volume, gamma, R, Q, variational
    #loss_weights = [5E-3, 1E-3, 5E-3, 5E-3, 5E-3, 5E-3, 1.]
    loss_weights = [0, 0, 0, 0, 0, 0, 1., 0.]
    #loss_weights = [0., 0., 0., 0., 0., 0., 1., 2E-4]
    #loss_weights = [5E-3, 1E-3, 5E-3, 5E-3, 5E-3, 5E-3, 0., 0.]

In [4]:
parameters = Defaults()
with open(f'params/{fname}.p', 'wb') as f:
    pickle.dump(parameters, f)

In [5]:
if parameters.deterministic:
    rng = np.random.RandomState(123456)
else:
    rng = np.random.RandomState()

# --- Instantiate environment ---
env = figure8_env(
    give_rewards=figure8_give_rewards,
    intern_dim=internal_dim,
    high_dim_obs=high_dim_obs,
    show_rewards=parameters.show_rewards
    )

# --- Instantiate learning_algo ---
learning_algo = CRAR(
    env,
    parameters.freeze_interval,
    parameters.batch_size,
    rng,
    high_int_dim=False,
    internal_dim=internal_dim, lr=parameters.learning_rate,
    nn_yaml=nn_yaml, double_Q=True,
    loss_weights=parameters.loss_weights,
    nstep=parameters.nstep, nstep_decay=parameters.nstep_decay,
    encoder_type=parameters.encoder_type,
    expand_tcm=parameters.expand_tcm
    )

if figure8_give_rewards:
    train_policy = EpsilonGreedyPolicy(
        learning_algo, env.nActions(), rng, 0.2,
        consider_valid_transitions=False
        )
    test_policy = EpsilonGreedyPolicy(
        learning_algo, env.nActions(), rng, 0.
        )
else:
    train_policy = FixedFigure8Policy.FixedFigure8Policy(
        learning_algo, env.nActions(), rng, epsilon=0.2,
        height=env.HEIGHT, width=env.WIDTH
        )
    test_policy = FixedFigure8Policy.FixedFigure8Policy(
        learning_algo, env.nActions(), rng,
        height=env.HEIGHT, width=env.WIDTH
        )

# --- Instantiate agent ---
agent = NeuralAgent(
    env, learning_algo,
    parameters.replay_memory_size,
    1, parameters.batch_size, rng,
    train_policy=train_policy, test_policy=test_policy)
if set_network is not None:
    agent.setNetwork(
        f'{set_network[0]}/fname', nEpoch=set_network[1],
        encoder_only=set_network[2]
        )

agent.run(10, 500)
print("end gathering data")

# --- Bind controllers to the agent ---
# Before every training epoch (periodicity=1), we want to print a summary of the agent's epsilon, discount and 
# learning rate as well as the training epoch number.
agent.attach(bc.VerboseController(
    evaluate_on='epoch', 
    periodicity=1))

# Learning rate may follow a scheduler
agent.attach(bc.LearningRateController(
    initial_learning_rate=parameters.learning_rate, 
    learning_rate_decay=parameters.learning_rate_decay,
    periodicity=1))

# During training epochs, we want to train the agent after every [parameters.update_frequency] action it takes.
# Plus, we also want to display after each training episode (!= than after every training) the average bellman
# residual and the average of the V values obtained during the last episode, hence the two last arguments.
agent.attach(bc.TrainerController(
    evaluate_on='action', 
    periodicity=parameters.update_frequency, 
    show_episode_avg_V_value=True, 
    show_avg_Bellman_residual=True))

# We wish to discover, among all versions of our neural network (i.e., after every training epoch), which one 
# has the highest validation score.
# To achieve this goal, one can use the FindBestController along with an InterleavedTestEpochControllers. It is 
# important that the validationID is the same than the id argument of the InterleavedTestEpochController.
# The FindBestController will dump on disk the validation scores for each and every network, as well as the 
# structure of the neural network having the best validation score. These dumps can then used to plot the evolution 
# of the validation and test scores (see below) or simply recover the resulting neural network for your 
# application.
agent.attach(bc.FindBestController(
    validationID=figure8_env.VALIDATION_MODE,
    testID=None,
    unique_fname=fname, savefrequency=5))

# All previous controllers control the agent during the epochs it goes through. However, we want to interleave a 
# "validation epoch" between each training epoch. For each validation epoch, we want also to display the sum of all 
# rewards obtained, hence the showScore=True. Finally, we want to call the summarizePerformance method of ALE_env 
# every [parameters.period_btw_summary_perfs] *validation* epochs.
agent.attach(bc.InterleavedTestEpochController(
    id=figure8_env.VALIDATION_MODE, 
    epoch_length=parameters.steps_per_test,
    periodicity=1,
    show_score=True,
    summarize_every=1,
    unique_fname=fname))

end gathering data


In [9]:
try:
    os.mkdir("params")
except Exception:
    pass
dump(vars(parameters), "params/" + fname + ".jldump")
#agent.gathering_data=False
if set_network is not None:
    agent.setNetwork(
        f'{set_network[0]}/fname', nEpoch=set_network[1],
        encoder_only=set_network[2]
        )
agent.run(parameters.epochs, parameters.steps_per_epoch)

# --- Show results ---
basename = "scores/" + fnamef
scores = load(basename + "_scores.jldump")
print(scores)

Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.0148,  0.1993,  0.1766, -0.1806,  0.1355, -0.2512,  0.0405,  0.0353,
         0.0724,  0.2387], device='cuda:0') tensor([ 0.2562, -0.1140,  0.1066, -0.4404,  0.0546,  0.1342, -0.2006,  0.2412,
         0.0801,  0.0411], device='cuda:0') tensor([ 0.0148,  0.1993,  0.1766, -0.1806,  0.1355, -0.2512,  0.0405,  0.0353,
         0.0724,  0.2387], device='cuda:0')
R[0]
tensor([-0.0099], device='cuda:0', grad_fn=<SelectBackward0>)
LOSSES
T = 0.049640529572963714; R = 0.0002913400748948334;                 Gamma = 0.5253344984650612; Q = 0.006225929114199971;
Entropy Neighbor = 0.9974085847139359;                 Entropy Random = 0.9959899883270263;                 Volume = 0.0; VAE = 0.0
Average (on the epoch) training loss: 0.006225929114199971
Episode average V value: 0.1380786164700985
epoch 1:
Learning rate: 0.0001
Discount factor: 0.9
Epsilon: 0.2


/home/cf2794/.conda/envs/auxrl/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Testing score per episode (id: 0) is 0.0 (average over 1 episode(s))
== Mean score per episode is 0.0 over 1 episodes ==


/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/figure8_env.py:365: RuntimeWarning: invalid value encountered in divide
  dist_matrix = dist_matrix/np.nanpercentile(dist_matrix.flatten(), 99)
/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/figure8_env.py:402: RuntimeWarning: All-NaN axis encountered
  ylim_max = np.nanmax(self._separability_tracking)*1.1


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([-0.5398,  0.5264,  0.3684, -0.4773, -0.3958, -0.2322, -0.5698,  0.1660,
         0.1926,  0.1535], device='cuda:0') tensor([-0.3077,  0.2204,  0.2747, -0.7481, -0.4670,  0.1646, -0.7977,  0.3728,
         0.2008, -0.0336], device='cuda:0') tensor([-0.5398,  0.5264,  0.3684, -0.4773, -0.3958, -0.2322, -0.5698,  0.1660,
         0.1926,  0.1535], device='cuda:0')
R[0]
tensor([0.0145], device='cuda:0', grad_fn=<SelectBackward0>)
LOSSES
T = 0.05249999190866947; R = 0.000492129858217595;                 Gamma = 0.5106260272860527; Q = 0.0063186622117846125;
Entropy Neighbor = 0.9991451889276505;                 Entropy Random = 0.9957783876657486;                 Volume = 0.0; VAE = 0.0
Average (on the epoch) training loss: 0.0063186622117846125
Episode average V value: 0.15547289729118347
epoch 2:
Learning rate: 0.0001
Discount factor: 0.9
Epsilon: 0.2
Tes

/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/figure8_env.py:365: RuntimeWarning: invalid value encountered in divide
  dist_matrix = dist_matrix/np.nanpercentile(dist_matrix.flatten(), 99)
/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/figure8_env.py:402: RuntimeWarning: All-NaN axis encountered
  ylim_max = np.nanmax(self._separability_tracking)*1.1


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
1 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.8512, -0.3743,  0.3597,  0.2599,  0.3244, -0.1007,  0.4285, -0.0552,
         0.0608, -0.1928], device='cuda:0') tensor([ 1.1404, -0.7183,  0.3650,  0.0255,  0.2757,  0.2852,  0.1718,  0.1151,
         0.1170, -0.4105], device='cuda:0') tensor([ 0.8515, -0.3746,  0.3591,  0.2604,  0.3254, -0.1004,  0.4285, -0.0552,
         0.0609, -0.1927], device='cuda:0')
R[0]
tensor([-0.0373], device='cuda:0', grad_fn=<SelectBackward0>)
LOSSES
T = 0.05573283176869154; R = 0.0010930867639835924;                 Gamma = 0.4433215583562851; Q = 0.0004937131118014122;
Entropy Neighbor = 0.9992015790939331;                 Entropy Random = 0.9940443394184113;                 Volume = 0.0; VAE = 0.0
Average (on the epoch) training loss: 0.0004937131118014122
Episode average V value: 0.15485129204392434
epoch 3:
Learning rate: 0.0001
Discount factor: 0.9
Epsilon: 0.2
T

/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/figure8_env.py:365: RuntimeWarning: invalid value encountered in divide
  dist_matrix = dist_matrix/np.nanpercentile(dist_matrix.flatten(), 99)
/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/figure8_env.py:402: RuntimeWarning: All-NaN axis encountered
  ylim_max = np.nanmax(self._separability_tracking)*1.1


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.7508, -0.2710,  0.4017,  0.1582,  0.1554, -0.1526,  0.4231, -0.0249,
         0.0326, -0.1862], device='cuda:0') tensor([ 0.9912, -0.6147,  0.3547, -0.1004,  0.0664,  0.2479,  0.1748,  0.1848,
         0.0668, -0.4015], device='cuda:0') tensor([ 0.7508, -0.2710,  0.4017,  0.1582,  0.1554, -0.1526,  0.4231, -0.0249,
         0.0326, -0.1862], device='cuda:0')
R[0]
tensor([-0.0238], device='cuda:0', grad_fn=<SelectBackward0>)



KeyboardInterrupt



## Visualize performance

In [ ]:
agent.setNetwork(fname, nEpoch=7)

In [ ]:
agent._in_episode = True
agent._mode = 0 # Testing mode with plan_depth=0
initState = env.reset(agent._mode)
inputDims = env.inputDimensions()

for i in range(len(inputDims)):
    if inputDims[i][0] > 1:
        agent._state[i][1:] = initState[i][1:]
agent._Vs_on_last_episode = []
is_terminal = False
reward = 0

### Frame by frame

In [ ]:
%matplotlib inline

for i in range(100):
    obs = env.observe()
    _obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
    plt.figure()
    plt.imshow(np.flip(_obs.squeeze()))
    plt.show()
    for i in range(len(obs)):
        agent._state[i][0:-1] = agent._state[i][1:]
        agent._state[i][-1] = obs[i]
    V, action, reward, _ = agent._step()
    print(action)
    agent._Vs_on_last_episode.append(V)
    is_terminal = env.inTerminalState()
    if is_terminal: break

### As animation

In [ ]:
%matplotlib notebook

import numpy as np 
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

fig, ax = plt.subplots(1,1)
obs = env.observe()
_obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
_obs = np.flip(_obs.squeeze())
ax.set_xticks([])
ax.set_yticks([])
im = ax.imshow(np.zeros(_obs.shape))

def init():
    plt.cla()
    im = ax.imshow(_obs)
    return [im]

def animate(i, *args, **kwargs):
    plt.cla()
    obs = env.observe()
    _obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
    _obs = np.flip(_obs.squeeze())
    im = ax.imshow(_obs)
    for i in range(len(obs)):
        agent._state[i][0:-1] = agent._state[i][1:]
        agent._state[i][-1] = obs[i]
        V, action, reward, _ = agent._step()
        agent._Vs_on_last_episode.append(V)
    return [im]

ani = animation.FuncAnimation(fig, animate, init_func=init, 
     frames=100, blit=False, repeat=True)
ani.save(f'figs/{fname}/behavior.gif', writer="ffmpeg", fps = 15)
fig.show()